In [12]:
from Bio import SeqIO
import pandas as pd
import numpy as np

from genominterv.remapping import remap
from genominterv.remapping import interval_distance, genomic
from genominterv.remapping import remap_interval_data
import seaborn as sns

Load file fasta fiel

In [35]:
file_path = "/home/johanulstrup/johan_gpn/people/johanulsrup/johan_gpn/data/macaca/macaca_GC_percent_chrx"

import pandas as pd

def load_variable_step_wig(file_path):
    data = []
    chrom = None
    span = 1

    with open(file_path, "r") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("track") or line.startswith("browser"):
                continue
            elif line.startswith("variableStep"):
                tokens = line.split()
                for token in tokens:
                    if token.startswith("chrom="):
                        chrom = token.split("=")[1]
                    elif token.startswith("span="):
                        span = int(token.split("=")[1])
            else:
                tokens = line.split()
                if len(tokens) != 2:
                    continue  # Skip malformed lines
                start, value = int(tokens[0]), float(tokens[1])
                end = start + span - 1
                data.append({
                    "chrom": chrom,
                    "start": start,
                    "end": end,
                    "value": value
                })

    return pd.DataFrame(data)

df_gc = load_variable_step_wig(file_path)  # or whatever the file is called

print(df_gc.head())
print(df_gc["chrom"].unique())
print(df_gc.shape)

  chrom  start  end  value
0  chrX      1    5   80.0
1  chrX      6   10   40.0
2  chrX     11   15   80.0
3  chrX     16   20   40.0
4  chrX     21   25  100.0
['chrX']
(100000, 4)


## loading eigenvectors using kasper code

In [14]:
def parse_compartment_data(file_name):
    e1_100kb = pd.read_csv(file_name)
    e1_100kb['start'] = [i*100_000 for i in range(e1_100kb.index.size)]
    e1_100kb['end'] = e1_100kb.start + 100_000
    e1_100kb['sign'] = np.sign(e1_100kb.e1)
    e1_100kb['segment_id'] = ((e1_100kb.sign.shift() != e1_100kb.sign)).cumsum()
    
    comp = e1_100kb.groupby('segment_id', as_index=False).agg(dict(
         e1=['mean', 'sum'], 
         start='min', 
         end='max', 
         segment_id='mean', 
         sign='mean'
    ))
    comp.columns = ['_'.join(col).strip() for col in comp.columns.values]
    comp = comp.rename(
        columns={'start_min':'start',
                 'end_max':'end', 
                 'segment_id_mean':'segment_id', 
                 'sign_mean':'sign'}
    )
    comp['comp'] = ['A' if x > 0 else 'B' for x in comp.sign]
    comp = comp.reset_index()
    comp['chrom'] = 'chrX'
    
    _comp = comp.copy()
    for i in range(1, _comp.index.size-1):
        if np.isnan(_comp.loc[i-1, 'e1_mean']):
            _comp.loc[i, 'start'] = np.nan
        if np.isnan(_comp.loc[i+1, 'e1_mean']):
            _comp.loc[i, 'end'] = np.nan
    _comp = _comp.loc[~_comp.e1_mean.isnull(), :]
    _comp = _comp.reset_index()
    compartment_edges = pd.concat([_comp.start, _comp.end]).sort_values().unique()
    
    compartments = comp.loc[~comp.e1_mean.isnull()].copy()
    compartments['start'] = compartments.start.astype(int)
    compartments['end'] = compartments.end.astype(int)

    return compartments, compartment_edges

def edge_segments(compartment_edges, flank):
    compartment_edge_segm = pd.DataFrame(np.column_stack((compartment_edges, compartment_edges+flank)), columns=['start', 'end'])
    compartment_edge_segm['chrom'] = 'chrX'
    return compartment_edge_segm

In [15]:
import os

# Load data
eigentrack_dir = "/home/johanulstrup/johan_gpn/people/johanulsrup/johan_gpn/data/eigentracks"
eigentrack_files = [
    f for f in os.listdir(eigentrack_dir) if f.endswith("_10Mb.csv")
]

comps_dict = {}
edges_dict = {}
generated_comps=[]
generated_edges=[]
a_and_b_comps = []

for filename in eigentrack_files:
    filepath = os.path.join(eigentrack_dir, filename)
    base = os.path.splitext(filename)[0]
    comps_var = f"{base}_comps"
    edges_var = f"{base}_edges"
    comps, edges = parse_compartment_data(filepath)
    comps_dict[comps_var] = comps
    edges_dict[edges_var] = edges
    generated_comps.append(comps_var)
    generated_edges.append(edges_var)

print("Generated variable names:", generated_comps)
print("Generated variable names:", generated_edges)

for edge_var in generated_edges:
    # Get the numpy array of edges from the dictionary
    edges = edges_dict[edge_var]
    
    # Create the DataFrame using edge_segments with flank=1
    seg_name = f"{edge_var}_interval"
    seg_df = edge_segments(edges, 1)
    #print(f"Created: {seg_name}")

    # Merge compartment assignment
    comps_var = edge_var.replace("_edges", "_comps")
    if comps_var in comps_dict:
        comps_df = comps_dict[comps_var]
        
        comp_df = pd.DataFrame({
            'comp': comps_df['comp'].reset_index(drop=True),
            'start': seg_df['start'].reset_index(drop=True),
            'end': seg_df['end'].reset_index(drop=True),
            'chrom': seg_df['chrom'].reset_index(drop=True)
        })

        # Save full merged comp_df
        comp_full_name = f"{edge_var}_interval_comp"
            # Save combined A and B compartments
        comp_AB_name = f"{edge_var}_AB"
        globals()[comp_AB_name] = comp_df
        a_and_b_comps.append(comp_AB_name)

        #print(f"Created: {comp_full_name}")

        # Split into compartments A and B
        comp_A = comp_df[comp_df['comp'] == 'A'].reset_index(drop=True)
        comp_B = comp_df[comp_df['comp'] == 'B'].reset_index(drop=True)

        # Save A and B splits as new variables
        comp_A_name = f"{edge_var}_A"
        comp_B_name = f"{edge_var}_B"
        globals()[comp_A_name] = comp_A
        globals()[comp_B_name] = comp_B
        a_and_b_comps.append(comp_A_name)
        a_and_b_comps.append(comp_B_name)
        
        #print(f"Created: {edge_var}_A and {edge_var}_B")

print("Generated compartment A and B variables:", a_and_b_comps)

#print(sperm_e1_100kb_10Mb_edges_A.head())

print(globals()[comp_AB_name].head())

Generated variable names: ['sperm_e1_100kb_10Mb_comps', 'round_spermatid_e1_100kb_10Mb_comps', 'pachytene_spermatocyte_e1_100kb_10Mb_comps', 'spermatogonia_e1_100kb_10Mb_comps', 'fibroblast_e1_100kb_10Mb_comps']
Generated variable names: ['sperm_e1_100kb_10Mb_edges', 'round_spermatid_e1_100kb_10Mb_edges', 'pachytene_spermatocyte_e1_100kb_10Mb_edges', 'spermatogonia_e1_100kb_10Mb_edges', 'fibroblast_e1_100kb_10Mb_edges']
Generated compartment A and B variables: ['sperm_e1_100kb_10Mb_edges_AB', 'sperm_e1_100kb_10Mb_edges_A', 'sperm_e1_100kb_10Mb_edges_B', 'round_spermatid_e1_100kb_10Mb_edges_AB', 'round_spermatid_e1_100kb_10Mb_edges_A', 'round_spermatid_e1_100kb_10Mb_edges_B', 'pachytene_spermatocyte_e1_100kb_10Mb_edges_AB', 'pachytene_spermatocyte_e1_100kb_10Mb_edges_A', 'pachytene_spermatocyte_e1_100kb_10Mb_edges_B', 'spermatogonia_e1_100kb_10Mb_edges_AB', 'spermatogonia_e1_100kb_10Mb_edges_A', 'spermatogonia_e1_100kb_10Mb_edges_B', 'fibroblast_e1_100kb_10Mb_edges_AB', 'fibroblast_e1_1

## not nesesary

In [16]:
for edge_var in generated_edges:
    comp_AB_name = f"{edge_var}_AB"
    if comp_AB_name in globals():
        print(f"Head of {comp_AB_name}:")
        print(globals()[comp_AB_name].head())
        print()

Head of sperm_e1_100kb_10Mb_edges_AB:
  comp       start         end chrom
0    A   2500000.0   2500001.0  chrX
1    A   8300000.0   8300001.0  chrX
2    A   9800000.0   9800001.0  chrX
3    A  10000000.0  10000001.0  chrX
4    B  10400000.0  10400001.0  chrX

Head of round_spermatid_e1_100kb_10Mb_edges_AB:
  comp      start        end chrom
0    A  7900000.0  7900001.0  chrX
1    A  8000000.0  8000001.0  chrX
2    A  8400000.0  8400001.0  chrX
3    B  8500000.0  8500001.0  chrX
4    A  8900000.0  8900001.0  chrX

Head of pachytene_spermatocyte_e1_100kb_10Mb_edges_AB:
  comp       start         end chrom
0    A   9000000.0   9000001.0  chrX
1    A   9100000.0   9100001.0  chrX
2    B   9800000.0   9800001.0  chrX
3    A  10000000.0  10000001.0  chrX
4    B  10500000.0  10500001.0  chrX

Head of spermatogonia_e1_100kb_10Mb_edges_AB:
  comp       start         end chrom
0    A   9700000.0   9700001.0  chrX
1    A  10000000.0  10000001.0  chrX
2    B  10100000.0  10100001.0  chrX
3    A  

## remapping


In [25]:
    # Clean and sort
sperm_clean = sperm_e1_100kb_10Mb_edges_AB.dropna(subset=["start", "end"]).copy()
sperm_clean["start"] = sperm_clean["start"].astype(int)
sperm_clean["end"] = sperm_clean["end"].astype(int)
df_gc = df_gc.dropna(subset=["start", "end"]).copy()
df_gc["start"] = df_gc["start"].astype(int)
df_gc["end"] = df_gc["end"].astype(int)

sperm_clean = sperm_clean.sort_values(by=['chrom', 'start', 'end'])
df_gc = df_gc.sort_values(by=['chrom', 'start', 'end'])

result = remap_interval_data(df_gc, sperm_clean, include_prox_coord=True)
# Post-process
result["mid"] = (result["start"] + result["end"]) / 2
result["absmid"] = result["mid"].abs()
#result = result.dropna(subset=["start", "end", "mid", "absmid", "start_prox", "end_prox"], how="any")

print(result.head())

print(result["start_prox"].unique())

     start      end  start_prox  end_prox chrom  start_orig  end_orig  value  \
0 -2499995 -2499999         NaN       NaN  chrX           1         5   80.0   
1 -2499990 -2499994         NaN       NaN  chrX           6        10   40.0   
2 -2499985 -2499989         NaN       NaN  chrX          11        15   80.0   
3 -2499980 -2499984         NaN       NaN  chrX          16        20   40.0   
4 -2499975 -2499979         NaN       NaN  chrX          21        25  100.0   

         mid     absmid  
0 -2499997.0  2499997.0  
1 -2499992.0  2499992.0  
2 -2499987.0  2499987.0  
3 -2499982.0  2499982.0  
4 -2499977.0  2499977.0  
[nan]
